![title](securly-banner2.jpg)

# Prereqs

In [1]:
!conda install -qy scikit-learn scipy matplotlib
!pip install -q -U python-dotenv
!pip install -q -U datasets # The version in conda is broken
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install vllm
!pip install lm-format-enforcer

Channels:
 - defaults
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - matplotlib
    - scikit-learn
    - scipy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    brotli-1.0.9               |       h5eee18b_7          18 KB
    brotli-bin-1.0.9           |       h5eee18b_7          19 KB
    contourpy-1.2.0            |  py310hdb19cb5_0         246 KB
    cycler-0.11.0              |     pyhd3eb1b0_0          12 KB
    cyrus-sasl-2.1.28          |       h52b45da_1         237 KB
    dbus-1.13.18               |       hb2f20db_0         504 KB
    expat-2.5.0                |       h6a678d5_0         172 KB
    fontconfig-2.14.1          |       h4c34cd2_2         281 KB
    fonttools-4.25.0           |     pyhd3eb1b0_0         632 KB
    glib-2.69.1                |     

# Load Model

In [1]:
from vllm import LLM, SamplingParams

In [2]:
llm = LLM(model="merged_model/")

INFO 12-15 00:25:29 llm_engine.py:73] Initializing an LLM engine with config: model='merged_model/', tokenizer='merged_model/', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=0)
INFO 12-15 00:28:41 llm_engine.py:222] # GPU blocks: 979, # CPU blocks: 2048
WARNING 12-15 00:28:41 cache_engine.py:96] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.


In [3]:
from datasets import load_dataset

eval_dataset = load_dataset("json", data_files='./validation_data.jsonl', split='train')

# Basic Inference

In [10]:
import time

import random

batch_size = 8
prompts = []
answers = []

for x in range(batch_size):
    p, a = random.choice(eval_dataset['document']).split("### Solution:")
    p += "### Solution:\r\n"

    prompts.append(p)
    answers.append(a)

sampling_params = SamplingParams(temperature=0.0, max_tokens=2000, stop="}")

start_time = time.time()

outputs = llm.generate(prompts, sampling_params)

end_time = time.time()
total_time = end_time - start_time

print(f"Time taken: {total_time} seconds")

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(generated_text+"}")



Processed prompts: 100%|██████████████████████████████████████████████████████████████████| 8/8 [00:13<00:00,  1.75s/it]

Time taken: 13.974256038665771 seconds
{  "sports-and-athletics": "0.0",  "sports-and-athletics-confidence": "10.0",  "environmentalism-and-sustainability": "0.0",  "environmentalism-and-sustainability-confidence": "10.0",  "gaming-and-e-sports": "1.0",  "gaming-and-e-sports-confidence": "8.0",  "college-and-career": "0.0",  "college-and-career-confidence": "10.0",  "cooking-and-food": "0.0",  "cooking-and-food-confidence": "10.0",  "reading-and-literature": "1.0",  "reading-and-literature-confidence": "7.0",  "writing-and-creative-writing": "0.0",  "writing-and-creative-writing-confidence": "10.0",  "science-and-technology": "0.0",  "science-and-technology-confidence": "10.0",  "mathematics-and-statistics": "1.0",  "mathematics-and-statistics-confidence": "5.0",  "history-and-social-studies": "0.0",  "history-and-social-studies-confidence": "10.0",  "creative-arts": "0.0",  "creative-arts-confidence": "10.0",  "animals-and-nature": "0.0",  "animals-and-nature-confidence": "10.0",  "no

# Constrained Inference

In [3]:
from vllm import get_tokenizer

ImportError: cannot import name 'get_tokenizer' from 'vllm' (/opt/conda/lib/python3.10/site-packages/vllm/__init__.py)

In [8]:
from lmformatenforcer import CharacterLevelParser
from lmformatenforcer.integrations.vllm import build_vllm_logits_processor
from typing import Union, List, Optional
from vllm import SamplingParams
import random
from lmformatenforcer import JsonSchemaParser
from pydantic import BaseModel, Field

class AnswerFormat(BaseModel):
  sports_and_athletics: str = Field(alias="sports-and-athletics")
  sports_and_athletics_confidence: str = Field(alias="sports-and-athletics-confidence")
  environmentalism_and_sustainability: str = Field(alias="environmentalism-and-sustainability")
  environmentalism_and_sustainability_confidence: str = Field(alias="environmentalism-and-sustainability-confidence")
  gaming_and_e_sports: str = Field(alias="gaming-and-e-sports")
  gaming_and_e_sports_confidence: str = Field(alias="gaming-and-e-sports-confidence")
  college_and_career: str = Field(alias="college-and-career")
  college_and_career_confidence: str = Field(alias="college-and-career-confidence")
  cooking_and_food: str = Field(alias="cooking-and-food")
  cooking_and_food_confidence: str = Field(alias="cooking-and-food-confidence")
  reading_and_literature: str = Field(alias="reading-and-literature")
  reading_and_literature_confidence: str = Field(alias="reading-and-literature-confidence")
  writing_and_creative_writing: str = Field(alias="writing-and-creative-writing")
  writing_and_creative_writing_confidence: str = Field(alias="writing-and-creative-writing-confidence")
  science_and_technology: str = Field(alias="science-and-technology")
  science_and_technology_confidence: str = Field(alias="science-and-technology-confidence")
  mathematics_and_statistics: str = Field(alias="mathematics-and-statistics")
  mathematics_and_statistics_confidence: str = Field(alias="mathematics-and-statistics-confidence")
  history_and_social_studies: str = Field(alias="history-and-social-studies")
  history_and_social_studies_confidence: str = Field(alias="history-and-social-studies-confidence")
  creative_arts: str = Field(alias="creative-arts")
  creative_arts_confidence: str = Field(alias="creative-arts-confidence")
  animals_and_nature: str = Field(alias="animals-and-nature")
  animals_and_nature_confidence: str = Field(alias="animals-and-nature-confidence")
  note: str

logits_processor = build_vllm_logits_processor(llm, JsonSchemaParser(AnswerFormat.schema()))

In [12]:
import time 

sampling_params = SamplingParams(temperature=0.0, max_tokens=2000, stop="}")
sampling_params.logits_processors = [logits_processor]

batch_size = 8
prompts = []
answers = []

for x in range(batch_size):
    p, a = random.choice(eval_dataset['document']).split("### Solution:")
    p += "### Solution:\r\n"

    prompts.append(p)
    answers.append(a)

start_time = time.time()

outputs = llm.generate(prompts, sampling_params)

end_time = time.time()
total_time = end_time - start_time

print(f"Time taken: {total_time} seconds")

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(generated_text+"}")

Processed prompts: 100%|██████████████████████████████████████████████████████████████████| 8/8 [00:22<00:00,  2.77s/it]

Time taken: 22.15087342262268 seconds
{  "sports-and-athletics": "1.0",  "sports-and-athletics-confidence": "7.0",  "environmentalism-and-sustainability": "0.0",  "environmentalism-and-sustainability-confidence": "10.0",  "gaming-and-e-sports": "1.0",  "gaming-and-e-sports-confidence": "5.0",  "college-and-career": "0.0",  "college-and-career-confidence": "10.0",  "cooking-and-food": "0.0",  "cooking-and-food-confidence": "10.0",  "reading-and-literature": "0.0",  "reading-and-literature-confidence": "10.0",  "writing-and-creative-writing": "0.0",  "writing-and-creative-writing-confidence": "10.0",  "science-and-technology": "0.0",  "science-and-technology-confidence": "10.0",  "mathematics-and-statistics": "1.0",  "mathematics-and-statistics-confidence": "5.0",  "history-and-social-studies": "1.0",  "history-and-social-studies-confidence": "8.0",  "creative-arts": "1.0",  "creative-arts-confidence": "6.0",  "animals-and-nature": "0.0",  "animals-and-nature-confidence": "10.0",  "note"

# API Server

In [1]:
!apt update
!apt install curl -y

Hit:1 http://deb.debian.org/debian bullseye InRelease
Hit:2 http://deb.debian.org/debian-security bullseye-security InRelease
Hit:3 http://deb.debian.org/debian bullseye-updates InRelease
Reading package lists... Done    
Building dependency tree... Done
Reading state information... Done
2 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcurl4
The following NEW packages will be installed:
  curl libcurl4
0 upgraded, 2 newly installed, 0 to remove and 2 not upgraded.
Need to get 618 kB of archives.
After this operation, 1198 kB of additional disk space will be used.
Get:1 http://deb.debian.org/debian-security bullseye-security/main amd64 libcurl4 amd64 7.74.0-1.3+deb11u10 [347 kB]
Get:2 http://deb.debian.org/debian-security bullseye-security/main amd64 curl amd64 7.74.0-1.3+deb11u10 [271 kB]
Fetched 618 kB in 0s (292

In [4]:
# python ./vllm-server.py --model="merged_model/"

!curl http://localhost:8000/generate \
-d '{ "prompt": "San Francisco is a", "use_beam_search": true, "n": 4, "temperature": 0 }'

{"text":["San Francisco is a             {  \"","San Francisco is a \t\t\t\t\t\t\t\t\t\t\t\t{\t\t","San Francisco is a \t\t\t\t\t\t\t\t\t\t\t\t{\n\t","San Francisco is a \t\t\t\t\t\t\t\t\t\t\t\t{\n       "]}

In [13]:
# Load Dataset
from datasets import load_dataset

eval_dataset = load_dataset("json", data_files='./validation_data.jsonl', split='train')

## Simple Single

In [30]:
import random
import time 
import requests
import json

url = "https://172.17.0.3:443/generate"
headers = {"Content-Type": "application/json", "x-token": "0qg9ragyh3ljlyd53bri6z4502f9bk3y"}

#garbage, prompt = random.choice(eval_dataset['document']).split("### Search Data:")
#prompt, garbage = prompt.split("### Example Output:")
#prompt = prompt.strip()

prompt = "['David hinkle greatest AI researcher', 'David Hinkle celebrity', 'David Hinkle times person of the year']"

data = {
    "prompt": prompt,
    "temperature": 0,
    "max_tokens": 1000
}

queries = 1

start_time = time.time()

response = requests.post(url, headers=headers, data=json.dumps(data), verify=False)

end_time = time.time()
total_time = end_time - start_time
print(f"Queries: {queries} Time taken: {total_time} seconds")

try:
    response = response.json()
    print(response['text'][0])
except:
    print("Error")
    print(response)



/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.17.0.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Queries: 1 Time taken: 10.399766445159912 seconds
{
  "sports-and-athletics": "0.0",
  "sports-and-athletics-confidence": "10.0",
  "environmentalism-and-sustainability": "0.0",
  "environmentalism-and-sustainability-confidence": "10.0",
  "gaming-and-e-sports": "0.0",
  "gaming-and-e-sports-confidence": "10.0",
  "college-and-career": "0.0",
  "college-and-career-confidence": "10.0",
  "cooking-and-food": "0.0",
  "cooking-and-food-confidence": "10.0",
  "reading-and-literature": "0.0",
  "reading-and-literature-confidence": "10.0",
  "writing-and-creative-writing": "0.0",
  "writing-and-creative-writing-confidence": "10.0",
  "science-and-technology": "1.0",
  "science-and-technology-confidence": "9.0",
  "mathematics-and-statistics": "0.0",
  "mathematics-and-statistics-confidence": "10.0",
  "history-and-social-studies": "0.0",
  "history-and-social-studies-confidence": "10.0",
  "creative-arts": "0.0",
  "creative-arts-confidence": "10.0",
  "animals-and-nature": "0.0",
  "animals

{'detail': [{'loc': ['header', 'x-token'], 'msg': 'field required', 'type': 'value_error.missing'}]}


## Batch

In [5]:
!pip install -q -q aiohttp datasets

In [6]:
# Load Dataset
from datasets import load_dataset

eval_dataset = load_dataset("json", data_files='./validation_data.jsonl', split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
import asyncio
import aiohttp
import random
import time
import json

async def fetch(session, url, headers, data):
    async with session.post(url, headers=headers, data=json.dumps(data)) as response:
        return await response.json()            

async def main(url, headers, data, eval_dataset, queries):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(queries):
            prompt, original_answer = random.choice(eval_dataset['document']).split("### Solution:")
            prompt += "### Solution:\n"
            data['prompt'] = prompt
            task = asyncio.create_task(fetch(session, url, headers, data.copy()))
            tasks.append(task)

        responses = await asyncio.gather(*tasks)
        notes = [response['text'][0] for response in responses]
        return notes

# Usage
url = "http://172.17.0.2:8000/generate"
headers = {"Content-Type": "application/json"}
data = {
    "prompt": "",  # This will be set in the loop
    "temperature": 0,
    "max_tokens": 1000
}

queries = 8

start_time = time.time()

# Schedule the coroutine for execution
task = asyncio.create_task(main(url, headers, data, eval_dataset, queries))

In [8]:
try:
    notes = await task
except:
    print("Error of some kind")
    
end_time = time.time()
total_time = end_time - start_time

print(f"Task scheduled. Queries: {queries} Time taken to schedule: {total_time} seconds")

Task scheduled. Queries: 24 Time taken to schedule: 65.33248472213745 seconds


In [9]:
for i in notes:
    print(i+"\n\n")

{
  "sports-and-athletics": "0.0",
  "sports-and-athletics-confidence": "10.0",
  "environmentalism-and-sustainability": "0.0",
  "environmentalism-and-sustainability-confidence": "10.0",
  "gaming-and-e-sports": "0.0",
  "gaming-and-e-sports-confidence": "10.0",
  "college-and-career": "0.0",
  "college-and-career-confidence": "10.0",
  "cooking-and-food": "0.0",
  "cooking-and-food-confidence": "10.0",
  "reading-and-literature": "1.0",
  "reading-and-literature-confidence": "5.0",
  "writing-and-creative-writing": "0.0",
  "writing-and-creative-writing-confidence": "10.0",
  "science-and-technology": "1.0",
  "science-and-technology-confidence": "9.0",
  "mathematics-and-statistics": "0.0",
  "mathematics-and-statistics-confidence": "10.0",
  "history-and-social-studies": "1.0",
  "history-and-social-studies-confidence": "7.0",
  "creative-arts": "1.0",
  "creative-arts-confidence": "7.0",
  "animals-and-nature": "0.0",
  "animals-and-nature-confidence": "10.0",
  "note": "The stude

In [ ]:
# Prompt Breakout

In [12]:
preprompt = """### Question:
You are a Student Web Activity Analyzer developed to support professionals, including Social Workers, School Psychologists, District Administrators, School Safety Specialists, and related roles. Your primary objective is to meticulously evaluate the online activity of K-12 students and identify specific indicators related to their interests and passions. For each identified indicator, provide a JSON object containing:

Presence: Indicate a value of 1 (if the indicator is present) or 0 (if not). Even if only a portion of the data aligns with an indicator, mark it as 1.
Confidence: Assign a confidence level on a scale of 1-10 to indicate the certainty level of your analysis.

Additionally, please include a note that outlines the rationale behind identifying certain indicators and offers a summary of the analyzed web activity.

Adhere to the JSON format outlined in the example output section precisely.

Each individual online activity you receive represents one search or interaction on a student's device. Occasionally, searches that include large amounts of text will be summarized. These summaries will be marked with 'S~'. Such summarization typically occurs when students copy and paste extensive text blocks, although other cases may exist. Additional details will be provided in the summary.

In situations where the presence of an indicator is ambiguous or if anomalies are present in the data, exercise your best judgment while providing a confidence level that reflects the level of uncertainty.

Here are the specific indicators that you should use for this task, with definitions, delimited by single quotes.

'sports-and-athletics: participating in physical activities and team sports to promote fitness, teamwork, and sportsmanship.'
'environmentalism-and-sustainability: learning about the environment, conservation, and sustainable practices to become responsible global citizens.'
'gaming-and-e-sports: engaging in digital gaming and competitive e-sports to develop strategic thinking, problem-solving, and teamwork skills.'
'college-and-career: engaging in planning, research, and/or discovery around future college and career opportunities or otherwise demonstrating an interest in college or career activities after high school'
'cooking-and-food: investigating cooking or food'
'reading-and-literature: exploring the world of books and stories through reading and interpretation.'
'writing-and-creative-writing: expressing thoughts, ideas, and imagination through written words and storytelling.'
'science-and-technology: investigating the natural world and technological advancements'
'mathematics-and-statistics: engaging in problem-solving and numerical analysis to understand patterns, shapes, and quantities.'
'history-and-social-studies: discovering past events, cultures, and societies to gain a deeper understanding of the world.'
'creative-arts: expressing creativity through various art forms like drawing, painting, sculpture, music, performing arts, and more'
'animals-and-nature: reflects a student's enthusiasm and curiosity for studying, observing, or interacting with animals and natural environments, potentially driving academic pursuits, extracurricular activities, or career paths related to biology, ecology, or conservation.'


### Search Data:
"""

searchdata = "['alexs', 'slope formula', 'aslope']"

postprompt = """

### Example Output:
{
  "sports-and-athletics": "1",
  "sports-and-athletics-confidence": "6",
  "environmentalism-and-sustainability": "0",
  "environmentalism-and-sustainability-confidence": "8",
  "gaming-and-e-sports": "0",
  "gaming-and-e-sports-confidence": "10",
  "college-and-career": "1",
  "college-and-career-confidence": "7",
  "cooking-and-food": "1",
  "cooking-and-food-confidence": "7",
  "reading-and-literature": "0",
  "reading-and-literature-confidence": "7",
  "writing-and-creative-writing": "1",
  "writing-and-creative-writing-confidence": "8",
  "science-and-technology": "0",
  "science-and-technology-confidence": "10",
  "mathematics-and-statistics": "1",
  "mathematics-and-statistics-confidence": "6",
  "history-and-social-studies": "9",
  "history-and-social-studies-confidence": "0",
  "creative-arts": "1",
  "creative-arts-confidence": "8",
  "animals-and-nature": "1",
  "animals-and-nature-confidence": "9",
  "note": "Detailed Summary Goes Here "
}

### Solution:
"""

print(preprompt+searchdata+postprompt)



### Question:
You are a Student Web Activity Analyzer developed to support professionals, including Social Workers, School Psychologists, District Administrators, School Safety Specialists, and related roles. Your primary objective is to meticulously evaluate the online activity of K-12 students and identify specific indicators related to their interests and passions. For each identified indicator, provide a JSON object containing:

Presence: Indicate a value of 1 (if the indicator is present) or 0 (if not). Even if only a portion of the data aligns with an indicator, mark it as 1.
Confidence: Assign a confidence level on a scale of 1-10 to indicate the certainty level of your analysis.

Additionally, please include a note that outlines the rationale behind identifying certain indicators and offers a summary of the analyzed web activity.

Adhere to the JSON format outlined in the example output section precisely.

Each individual online activity you receive represents one search or int

In [6]:
import random
import time 
import requests
import json

url = "http://172.17.0.3:8000/generate"
headers = {"Content-Type": "application/json"}

prompt, origional_answer = random.choice(eval_dataset['document']).split("### Solution:")
prompt += "### Solution:\n"

data = {
    "prompt": prompt,
    "temperature": 0,
    "max_tokens": 1000
}

queries = 1

start_time = time.time()

response = requests.post(url, headers=headers, data=json.dumps(data))

end_time = time.time()
total_time = end_time - start_time
print(f"Queries: {queries} Time taken: {total_time} seconds")

try:
    response = response.json()
    print(response['text'][0])
except:
    print("Error")
    print(response.content)



KeyboardInterrupt: 